In [14]:

!python3 -c "import sys; print(sys.executable)"
!ls

/home/sabih/anaconda3/bin/python3
conda.yml  mlflow.db  mlflow_env_vars.sh  MLFlow_Lab.ipynb  MLproject  train.py


# MLFlow lab

In [17]:
# !pip install pandas
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [3]:
# !pip install mlflow

In [18]:
pd.__version__

NameError: name 'pd' is not defined

### Setting up MLFlow tracking server

We also specify artifact root and backend store URI. This makes it possible to store models.

After running this command tracking server will be accessible at `localhost:5000`

In [19]:
%%bash --bg

mlflow server --host 0.0.0.0 \
    --port 5005 \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./mlruns

### MLProject file

This file is used to configure MLFlow steps.

Using `MLproject` we can define our project's pipeline steps, called *entry points*.

Each entry point in this file corresponds to a shell command.

Entry points can be ran using

```
mlflow run -e <ENTRY_POINT>
```

By default `mlflow run` runs `main` entrypoint.

In [20]:
%cat MLproject

name: basic_mlflow

# this file is used to configure Python package dependencies.
# it uses Anaconda, but it can be also alternatively configured to use pip.
conda_env: conda.yml

# entry points can be ran using `mlflow run <project_name> -e <entry_point_name>
entry_points:
  # download_data:
    # you can run any command using MLFlow
    # command: "bash download_data.sh"
  # MLproject file has to have main entry_point. It can be toggled without using -e option.
  main:
    # parameters is a key-value collection.
    parameters:
      file_name:
        type: str
        default: "day.csv"
      max_k:
        type: int
        default: 10
    command: "python train.py"


First we need to download data. We will use weather data from previous machine learning tutorial.

In [8]:
%%bash
source mlflow_env_vars.sh
mlflow run .  -e download_data

Traceback (most recent call last):
  File "/home/sabih/.local/bin/mlflow", line 8, in <module>
    sys.exit(cli())
  File "/usr/lib/python3/dist-packages/click/core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/click/core.py", line 1053, in main
    rv = self.invoke(ctx)
  File "/usr/lib/python3/dist-packages/click/core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/lib/python3/dist-packages/click/core.py", line 1395, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/lib/python3/dist-packages/click/core.py", line 754, in invoke
    return __callback(*args, **kwargs)
  File "/home/sabih/.local/lib/python3.10/site-packages/mlflow/cli.py", line 202, in run
    projects.run(
  File "/home/sabih/.local/lib/python3.10/site-packages/mlflow/projects/__init__.py", line 338, in run
    submitted_run_obj = _run(
  File "/home/sabih/.local/lib/python3.10/site-packa

CalledProcessError: Command 'b'source mlflow_env_vars.sh\nmlflow run .  -e download_data\n'' returned non-zero exit status 1.

## Training

Now we can train models. See `train.py`.
It contains code from supervised machine learning tutorial; we added tracking metrics and model.

We will train kNN models for $k \in \{1, 2, ..., 10\}$ using *temperature* and *casual* features.

After running this command you can go to `localhost:5000` and see the trained models.

In [21]:
import sklearn

ModuleNotFoundError: No module named 'sklearn'

In [11]:
sklearn.__version__

  Using cached fire-0.5.0-py2.py3-none-any.whl
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)


In [22]:
%%bash
source mlflow_env_vars.sh
mlflow run .

Traceback (most recent call last):
  File "/home/sabih/.local/bin/mlflow", line 8, in <module>
    sys.exit(cli())
  File "/usr/lib/python3/dist-packages/click/core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/click/core.py", line 1053, in main
    rv = self.invoke(ctx)
  File "/usr/lib/python3/dist-packages/click/core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/lib/python3/dist-packages/click/core.py", line 1395, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/lib/python3/dist-packages/click/core.py", line 754, in invoke
    return __callback(*args, **kwargs)
  File "/home/sabih/.local/lib/python3.10/site-packages/mlflow/cli.py", line 202, in run
    projects.run(
  File "/home/sabih/.local/lib/python3.10/site-packages/mlflow/projects/__init__.py", line 338, in run
    submitted_run_obj = _run(
  File "/home/sabih/.local/lib/python3.10/site-packa

CalledProcessError: Command 'b'source mlflow_env_vars.sh\nmlflow run .\n'' returned non-zero exit status 1.

## Inspecting stored models

The trained models are stored in `mlruns/0`.

These directories contain artifacts and config that is needed to serve them.

In [13]:
%%bash
last_model_path=$(ls -tr mlruns/0/ | tail -1)
cat mlruns/0/$last_model_path/artifacts/knn/MLmodel

artifact_path: knn
flavors:
  python_function:
    env:
      conda: conda.yaml
      virtualenv: python_env.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.11.3
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.2.2
mlflow_version: 2.3.1
model_uuid: dba0f8dd732b4ab1b1d16ae785940c78
run_id: 9d54d157e9e24ad1b7e63f94519db5e9
utc_time_created: '2023-05-02 11:26:23.769785'


In [14]:
import mlflow

In [15]:
mlflow.__version__

'2.3.1'

## Serving model

Now that we trained our models we can go to *Models* page on MLFLow UI (http://localhost:5000/#/models).

Click *sklearn_knn* on this page, choose a model and move it to *Production* stage.

The following cell will serve the model at localhost on port 5001.

In [33]:
%%bash --bg
source mlflow_env_vars.sh
mlflow --version
mlflow models serve -m models:/sklearn_knn/Production -p 5001 --env-manager=conda 


# Prediction

We'll load data that we can feed into prediction server.

In [34]:
df = pd.read_csv("day.csv")[["temp", "casual", "season"]]
df["is_winter"] = df["season"] == 1

df[~df["is_winter"]].head()

,temp,casual,season,is_winter
79,0.430435,401,2,False
80,0.441667,460,2,False
81,0.346957,203,2,False
82,0.285000,166,2,False
83,0.264167,300,2,False


Let's predict for first winter day and first non-winter day (first rows of previous two dataframes)

**warning: this might fail at first because the prediction server didn't spin up; in this case wait a minute**

In [35]:
%%bash
data='[[0.344,331], [0.43, 401]]'
echo $data

curl -d "{\"inputs\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[0.344,331], [0.43, 401]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    68  100    30  100    38   6303   7984 --:--:-- --:--:-- --:--:-- 17000


{"predictions": [true, false]}

In [36]:
%%bash
data='[[0.344,331], [0.43, 401]]'
echo $data

curl -d "{\"instances\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[0.344,331], [0.43, 401]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    71  100    30  100    41  12897  17626 --:--:-- --:--:-- --:--:-- 35500


{"predictions": [true, false]}

In [38]:
%%bash
data='[[0.344,331], [0.43, 401]]'
columns='["temp","casual"]'
echo $data

curl -d "{\"dataframe_split\":{\"columns\":[\"temp\",\"casual\"],\"data\": $data}}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[0.344,331], [0.43, 401]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   114  100    30  100    84   2856   7999 --:--:-- --:--:-- --:--:-- 11400


{"predictions": [true, false]}

Voila! We see that the model outputs correct predictions.